In [1]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

In [28]:
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) / 2, (ptA[1] + ptB[1]) / 2)

In [29]:
image_path = 'CB.JPEG'
width_reference_object = 1

In [30]:
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7,7), 0)

In [31]:
edged = cv2.Canny(gray, 50, 100)
edged = cv2.dilate(edged, None, iterations= 1)
edged = cv2.erode(edged, None, iterations= 1)

In [32]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

In [33]:
# sort the contour from left to right
# initialize the color
# init the reference object - which will store bounding box, centroid, pixels-per-metric
(cnts, _) = contours.sort_contours(cnts, 'left-to-right')
colors = ((0, 0, 255), (240, 0, 159), (0, 165, 255), (255, 255, 0), (255, 0, 255))
refObj = None

In [34]:
for c in cnts:

    if cv2.contourArea(c) < 500:
        continue

    # compute the rotated bounding box of the contour
    box = cv2.minAreaRect(c)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    box = np.array(box, dtype="int")

    # order the points in the contour such that they appear
    # in top-left, top-right, bottom-right, and bottom-left
    # order, then draw the outline of the rotated bounding box
    box = perspective.order_points(box)

    #compute the center of the bounding box
    cX = np.average(box[:, 0])
    cY = np.average(box[:, 1])

    # if this is the first contour we are examining (i.e.,
    # the left-most contour), we presume this is the
    # reference object
    if refObj is None:
        # unpack the ordered bounding box, then compute the
        # midpoint between the top-left and top-right points,
        # followed by the midpoint between the top-right and
        # bottom-right
        (tl, tr, br, bl) = box
        (tlblX, tlblY) = midpoint(tl, bl)
        (trbrX, trbrY) = midpoint(tr, br)

        # compute the Euclidean distance between the midpoints,
        # then construct the reference object
        D = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
        refObj = (box, (cX, cY), D/width_reference_object)
        continue

    # draw the contours on the image
    orig = image.copy()
    cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
    cv2.drawContours(orig, [refObj[0].astype("int")], -1, (0, 255, 0), 2)

    # stack the reference coordinates and the object coordinates
    # to include the object center
    refCoords = np.vstack([refObj[0], refObj[1]])
    objCoords = np.vstack([box, (cX, cY)])

    # loop over the original points
    for ((xA, yA), (xB, yB), color) in zip(refCoords, objCoords, colors):
        # draw circles corresponding to the current points and
        # connect them with a line
        cv2.circle(orig, (int(xA), int(yA)), 5, color, -1)
        cv2.circle(orig, (int(xB), int(yB)), 5, color, -1)
        cv2.line(orig, (int(xA), int(yA)), (int(xB), int(yB)), color, 2)
        # compute the Euclidean distance between the coordinates,
        # and then convert the distance in pixels to distance in
        # units
        D = dist.euclidean((xA, yA), (xB, yB)) / refObj[2]
        (mX, mY) = midpoint((xA, yA), (xB, yB))
        cv2.putText(orig, "{:.1f}in".format(D), (int(mX), int(mY - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)
        # show the output image
        cv2.imshow("Image", orig)
        cv2.waitKey(0)

KeyboardInterrupt: 

In [2]:
from tkinter import *
from PIL import ImageTk, Image
import math

root = Tk()

img = ImageTk.PhotoImage(Image.open("CB.JPEG"))
panel = Label(root, image=img)
panel.pack(side="bottom", fill="both", expand="yes")
x0 = 0
y0 = 0
x1 = 0
y1 = 0


def leftClick(event):
    global x0, y0
    x0 = event.x
    y0 = event.y
    # return [x0, y0]


panel.bind("<Button-1>", leftClick)


def rightClick(event):
    global x1, y1
    x1 = event.x
    y1 = event.y
    # return x1, y1


panel.bind("<Button-3>", rightClick)


def getDistance(event):
    global x0, y0, x1, y1
    distance = math.sqrt(((x0 - x1)**2)+((y0 - y1)**2))
    print(distance)


panel.bind("<Button-2>", getDistance)
root.mainloop()

9.848857801796104
9.848857801796104
9.848857801796104
9.848857801796104
9.848857801796104


In [6]:
from tkinter import *
from PIL import ImageTk, Image
import math

root = Tk()

img = ImageTk.PhotoImage(Image.open("CB.JPEG"))
panel = Label(root, image=img)
panel.pack(side="bottom", fill="both", expand="yes")
counter = 0
x0 = 0
x1 = 0
y0 = 0
y1 = 0


def getDistance(event):
    global counter, x0, y0, x1, y1
    if counter == 0:
        x0 = event.x
        y0 = event.y
        counter += 1
    elif counter == 1:
        x1 = event.x
        y1 = event.y
        counter += 1
    elif counter == 2:
        distance = math.sqrt(((x0 - x1)**2)+((y0 - y1)**2))
        print(distance)
        counter = 0


panel.bind("<Button-1>", getDistance)
root.mainloop()

645.3100030218035


In [7]:
import tkinter as tk
from PIL import ImageTk, Image
import math
start_point_x, start_point_y, end_point_x, end_point_y = 0, 0, 0, 0

def mouse_left_down_detection(event):
    global start_point_x, start_point_y
    start_point_x = event.x
    start_point_y = event.y

def mouse_left_release_detection(event):
    global end_point_x, end_point_y
    end_point_x = event.x
    end_point_y = event.y
    print(start_point_x, start_point_y, end_point_x, end_point_y)
    print(get_instance(start_point_x, start_point_y, end_point_x, end_point_y))

def get_instance(x1, y1, x2, y2):
    return math.sqrt((pow(abs(x2-x1), abs(x2-x1))+pow(abs(y2-y1), abs(y2-y1))))

image_path = "CB.JPEG"
root = tk.Tk()
img = ImageTk.PhotoImage(Image.open(image_path))
panel = tk.Label(root, image=img)
# Bind event mouse left down
panel.bind("<Button-1>", mouse_left_down_detection)
# Bind event mouse left release and calculate distance
panel.bind("<ButtonRelease-1>", mouse_left_release_detection)
panel.pack(side="bottom", fill="both", expand="yes")
root.mainloop()

109 149 109 149
1.4142135623730951
89 794 89 794
1.4142135623730951
109 145 481 795


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Joe\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-7-27701e5c4c98>", line 16, in mouse_left_release_detection
    print(get_instance(start_point_x, start_point_y, end_point_x, end_point_y))
  File "<ipython-input-7-27701e5c4c98>", line 19, in get_instance
    return math.sqrt((pow(abs(x2-x1), abs(x2-x1))+pow(abs(y2-y1), abs(y2-y1))))
OverflowError: int too large to convert to float


485 795 485 795
1.4142135623730951


In [24]:
import numpy as np
import cv2
import math
events = [i for i in dir(cv2) if 'EVENT' in i]
print(events)

['EVENT_FLAG_ALTKEY', 'EVENT_FLAG_CTRLKEY', 'EVENT_FLAG_LBUTTON', 'EVENT_FLAG_MBUTTON', 'EVENT_FLAG_RBUTTON', 'EVENT_FLAG_SHIFTKEY', 'EVENT_LBUTTONDBLCLK', 'EVENT_LBUTTONDOWN', 'EVENT_LBUTTONUP', 'EVENT_MBUTTONDBLCLK', 'EVENT_MBUTTONDOWN', 'EVENT_MBUTTONUP', 'EVENT_MOUSEHWHEEL', 'EVENT_MOUSEMOVE', 'EVENT_MOUSEWHEEL', 'EVENT_RBUTTONDBLCLK', 'EVENT_RBUTTONDOWN', 'EVENT_RBUTTONUP']


In [58]:
import numpy as up
from numpy import linalg as LA

first = np.zeros((2,1))
second = np.zeros((2,1))
print(first)

def click_event(event,x,y,flags,param):
    global first,second
    if event == cv2.EVENT_LBUTTONDOWN:
        print('(',x,' , ',y,')')
        font = cv2.FONT_HERSHEY_SIMPLEX
        strXY = str(x) + ',' + str(y)
        cv2.putText(img, strXY,(x,y),font,1,(225,225,0),3)
        cv2.imshow('image',img)
        first[0] = float(x)
        first[1] = float(y)
        print(first)
    if event ==cv2.EVENT_RBUTTONDOWN:
        print('(',x,' , ',y,')')
        font = cv2.FONT_HERSHEY_SIMPLEX
        strXY = str(x) + ',' + str(y)
        cv2.putText(img, strXY,(x,y),font,1,(0,0,225),3)
        cv2.imshow('image',img)
        second[0] = float(x)
        second[1] = float(y)
        print(second)
        
        dist = LA.norm(first - second)
        print(dist)
        strDist = 'distance is :' + str(dist)
        cv2.putText(img, strDist,(x+50,y+50),font,1,(0,0,225),3)
        cv2.imshow('image',img)
        #d = get_dinstance(x_1, y_1, x_2,y_2)
        
        #dis = str(d)
        
        
        #cv2.putText(img,dis,d,font,1,(0,225,0),3)
        #cv2.imshow('image',img)

        


#print(get_instance(x_1,y_1,x_2,y_2))
img = cv2.imread('CB.JPEG')

cv2.imshow('image',img)
cv2.setMouseCallback('image',click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[0.]
 [0.]]
( 115  ,  149 )
[[115.]
 [149.]]
( 89  ,  792 )
[[ 89.]
 [792.]]
643.5254462723288
( 104  ,  382 )
[[104.]
 [382.]]
( 96  ,  587 )
[[ 96.]
 [587.]]
205.15603817582362
( 322  ,  386 )
[[322.]
 [386.]]
( 662  ,  394 )
[[662.]
 [394.]]
340.0941046239996
( 317  ,  228 )
[[317.]
 [228.]]
( 640  ,  238 )
[[640.]
 [238.]]
323.154761685481
( 312  ,  79 )
[[312.]
 [ 79.]]
( 711  ,  85 )
[[711.]
 [ 85.]]
399.0451102319135
